# SILVA-CRAS

# Capitulo 1 - Articulación SMByC y especialista SIG PatNat
# Modulo Detección de cambios por deforestación.

# Proyecto: Sistema para la toma de decisiones en el marco de los acuerdos de conservación de bosques a nivel predial en el piedemonte Caquetá, Programa Conservación y Gobernanza-Patrimonio Natural 


Josef Kellndorfer, Ph.D., President and Senior Scientist, Earth Big Data, LLC

Carlos Pedraza, Ms.C., Spatial analyst and Monitoring Expert, iScioLab and Earth Big Data, LLC

Cristhian Fabian Forero, Remote Sensing Expert, iScioLab.

Fecha revisión: Pendiente

Este capitulo de análisis de imágenes SAR para aplicaciones forestales esta enfocado al monitoreo de la detección de cambio por deforestación como un notebook interactivo dirigido al perfil de usuario de especialista SIG del programa C&G y a la estrategía de articulación con instituciones del gobierno, como lo es el Sistema de Minitoreo de Bosques y Carbono de Colombia (SMByC). El formato difgital (*jupyter notebook*) de este capitulo puede ser ejecutado desde cualquier explorador para la exploración de los datos historicos y nuevos que hacen parte del sistema de toma de decisiones en el marco de acuerdos de conservación a nivel predial del programa Conservación y Gobernanza de Patrimonio Natural. El notebook esta construido a partir de textos, documentación, código de ejecución de Python y formato de reducción de marcado, incluyendo ecuaciones matemáticas de estilo látex. Con este enfoque, el usuario puede ampliar, cambiar y compartir fácilmente todo el trabajo con nuevos conjuntos de datos en nuevas regiones o nuevos pasos de series temporales disponibles. 

## Instalación de Software y Datos

Por favor referirse a los documentos de **INSTALACIÓN** y **DATA-HOW TO**.

Los datos de series de tiempo fueron pre-procesados con ***S**oftware for **E**arth Big Data **P**rocessing, **P**rediction Modeling and **O**rganization* (SEPPO) EARTH BIG DATA utilizando rutinas de procesamiento basadas en la nube a través de Amazon Web Services (AWS). SEPPO permite realizar el procesamiento completo de grandes volumenes de datos SAR (y otros productos de sensores remotos) para construir datos de series de tiempo eficeintemente. La gía de formatos de datos **EBD_README** explica la estructura de los datos y las convenciones de nomnres de los datos producidos por EARTH BIG DATA, LLC.


# Notas de como trabajar con este Notebook

1. Después de iniciar el servidor Notebooks y abrir un Notebook, navega al menú **Kernel** y elige ebd:

    > Kernel > Change Kernel > Python \[conda env:ebd\]

2. Para ejecutar el código en una celda, posicione su cursor parpadeante dentro de una celda y seleccione el botón *Run* de la barra de menú del notebook, o use la siguiente combinación de teclas:
    - CTRL+Enter para ejecutar la celda
    - ALT+Enter para ejecutar una celda e insertar una nueva celda debajo

3. Para comentar las líneas dentro de las celdas del código use como primer carácter *#*. Puede marcar varias líneas y usar una combinación de teclas para comentar el bloque con:
    - Windows: CTRL+/
    - MacOS:  CMD+/

# Importando paquetes de Python
El primer paso en el enfoque de análisis de series de tiempo después de obtener las conjuntos de datos preprocesados es la importación de los paquetes de Python necesarios.
Véanse los comentarios que figuran a continuación sobre los paquetes necesarios y sus funciones. Obsérvese que todos estos paquetes deberían haberse instalado cuando se creó el entorno de la Anaconda Python. 

In [1]:
import pandas as pd
import gdal
import numpy as np
import time,os, glob

# Setup plotting inside the notebook
%matplotlib inline
import matplotlib.pylab as plt

# Definiendo directorio del proyecto y nombre de archivos
Revisa [EBD Guía de datos](https://github.com/EarthBigData/openSAR/blob/master/documentation/EBD_DataGuide.md) para una explicación de las convenciones de los nombres de archivos utilizados para los datos de imágenes.

 
#### Como especificar los nombres de directorios:

**Linux**: /path/to/file

**Windows**: d:/path/to/file       
    d: es la letra del drive   # IMPORTANTE: Siempre utiliza '/' en vez de '\' en Windows

NOTA: directorios y nombres de archivos estan especificados en python como cadenas encerradas por comillas simples o dobles:'string' "string"

## Caquetá MGRS Tile 18NWF

In [2]:
datadirectory='/Users/amenaza/github/Silva-Cras-PatNat/Data/Colombia/S32618X557000Y64000sS1_EBD'
datefile='S32618X557000Y64000sS1_D_vv_0069_mtfil.dates'
imagefile='S32618X557000Y64000sS1_D_vv_0069_mtfil.vrt'
imagefile_cross='S32618X557000Y64000sS1_D_vh_0069_mtfil.vrt'

## Cambiar al directorio de datos

In [3]:
os.chdir(datadirectory)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/amenaza/github/Silva-Cras-PatNat/Data/Colombia/S32618X557000Y64000sS1_EBD'

In [ ]:
os.getcwd()  # Muestra el directorio de datos

In [ ]:
glob.glob("*.vrt")   # Lista los datos disponibles en el directorio de datos

# Fechas de adquisición
## Lee de los datos las fechas de las series de tiempo y crea un indice de fechas Panda

In [ ]:
# Lee datos de las fechas de las series de tiempo
# y crea un índice de fechas en Panda
dates=open(datefile).readlines() 
tindex=pd.DatetimeIndex(dates)

In [ ]:
# A partir del índice se construye y despliega una lookup table para 
# para los números asociados a las bandas y fechas 
j=1
print('Bands and dates for',imagefile)
for i in tindex:
    print("{:4d} {}".format(j, i.date()),end=' ')
    j+=1
    if j%5==1: print()

# Datos de imagen

Para abrir **open** un archivo de imagen y hacerlo legible se aplica la función gdal.Open(). Esto transforma el tipo de imagen para que pueda ser usada en futuras funciones con este archivo:

In [ ]:
img=gdal.Open(imagefile)

## Acá vamooooooooosssssssssss
To explore the image, e.g. number of bands, pixels, lines you can use several functions associated with the openend image object, e.g.:

In [ ]:
print(img.RasterCount) # Number of Bands
print(img.RasterXSize) # Number of Pixels
print(img.RasterYSize) # Number of Lines
print (gdal.Info(imagefile)) # Image parametres

## Leyendo loda datos de una banda de la imágen
To access any band in the image, use the img.GetRasterBand(x) function. E.g. to access the first band x=1, the last band: x=60. 

In [ ]:
band=img.GetRasterBand(1) # gdal understan the 1st band as 1, an Python as 0

Once a band is seleted, several functions associated with the band are available for further processing, e.g.

* band.ReadAsArray(xoff=0,yoff=0,xsize=None,ysize=None)

So, to read the entire raster layer for the band:

In [ ]:
raster=band.ReadAsArray()
raster[raster==0]=1    # Lazy dealing with no data values here

## Subsets
Because of the potentially large data volume when dealing with time series data stacks, it may be required to read only a subset of data. 

With the gdal .ReadAsArray() function, subsets can be requested with offsets and size:

**img.ReadAsArray(xoff=0, yoff=0, xsize=None, ysize=None)**

xoff,yoff are the offsets from the upper left corner in pixel/line coordinates. 

xsize,ysize specify the size of the subset in x-direction (left to right) and y-direction (top to bottom).

E.g., to read only a **subset** of 5x5 pixels with an offset of 5 pixels and 20 lines:

In [ ]:
raster_sub=band.ReadAsArray(5,20,5,5)
raster_sub[raster_sub==0]=1

The result is a two dimensional numpy array with the datatpye the data were stored in. We can inspect these data in python by simply typing the array name on the commandline:

In [ ]:
raster_sub

## Displaying Bands in the Time Series of SAR Data
From the lookup table we know that bands 20 and 27 in the HKH Center dataset are from late February and late August. Let's take look at these images.

**HINT: Because python is an object oriented scripting language, we can often combine several steps (or function calls) into one command. See the trick below to access a raster band and read the data in one step.**

In [ ]:
raster_1 = img.GetRasterBand(20).ReadAsArray()
raster_2 = img.GetRasterBand(27).ReadAsArray()

## Plotting in Python to Visualize the Image Bands
Matplotlib's plotting functions allow for powerful options to display imagery. We are following some standard approaches for setting up figures.
First we are looking at a **raster band** and it's associated **historgram**.

In [ ]:
fig = plt.figure(figsize=(16,8)) # Initialize figure with a size
ax1 = fig.add_subplot(121)  # 121 determines: 1 row, 2 plots, first plot
ax2 = fig.add_subplot(122)  # 122 determines: 1 row, 2 plots, second plot

# First plot: Image
bandnbr=20
ax1.imshow(raster_1,cmap='gray',vmin=2000,vmax=10000,interpolation='bilinear')
ax1.set_title('Image Band {} {}'.format(bandnbr,
                                    tindex[bandnbr-1].date()))

# Second plot: Historgram
# IMPORTANT: To get a histogram, we first need to *flatten* 
# the two-dimensional image into a one-dimensional vector.
h = ax2.hist(raster_1.flatten(),bins=100,range=(0,10000))
ax2.xaxis.set_label_text('Amplitude (Uncalibrated DN Values)')
_=ax2.set_title('Histogram Band {} {}'.format(bandnbr,
                                    tindex[bandnbr-1].date()))

### Writing a plotting function for the task
Below, the plotting commands used above are **def**ined in a function named *showImage*. Several parameters can be passed to the function, some with default values listed at the end:

- raster = a numpy two dimensional array 
- tindex = a panda index array for dates
- bandnbr = the band number the corresponds to the raster 
- vmin = minimim value to display 
- vmax = maximum value to display

Note: By default, data will be linearly stretched between vmin and vmax. We also provide a interpolation option for the plotting routine that accepts values like 'nearest', 'bilinear', 'bicubic'. We set this parameter by default to 'bilinear'.

In [ ]:
def showImage(raster,tindex,bandnbr,vmin=None,vmax=None,interpolation='bilinear'):
    fig = plt.figure(figsize=(16,8))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)

    ax1.imshow(raster,cmap='gray',vmin=vmin,vmax=vmax,interpolation='bilinear')
    ax1.set_title('Image Band {} {}'.format(bandnbr,
                                tindex[bandnbr-1].date()))
    vmin=np.percentile(raster,2) if vmin==None else vmin
    vmax=np.percentile(raster,98) if vmax==None else vmax
    ax1.xaxis.set_label_text(
        'Linear stretch Min={} Max={}'.format(vmin,vmax))
    
    
    h = ax2.hist(raster.flatten(),bins=100,range=(0,11000))
    ax2.xaxis.set_label_text('Amplitude (Uncalibrated DN Values)')
    ax2.set_title('Histogram Band {} {}'.format(bandnbr,
                                tindex[bandnbr-1].date()))

## Time Series Data Stacks

Just as we can use the ReadAsArray() function on a band, we can actually use it on the entire image data stack. To read an entire stack, i.e. all bands use the function on the image data handle:

> img.ReadAsArray()

CAUTION: Since this could potentially result in large memory need, it is wise to do some preliminary calcuations as to how large of a data set would be read in. for that we can do the following calculation:

${Data 
Volume [GB]} = \frac{ N_{bands} \times N_{pixels} \times N_{lines} \times {Bytes_{pixel}}}{{1024}^3}$
    
For SAR data we typically use dataypes of 

* Float 32 bit (4 bytes per pixel) for power and dB data, 
* Unsigned Integer 16 bit (2 bytes per pixel) linearly scaled amplitudes, and
* Unsigned Byte (1 byte per pixel) for dB-scaled to 8 bit data 

The following table gives an overview of typically used data types for SAR data analysis in python:

| Data Type  	|   Numpy Name	|   GDAL Name	|   GDAL Code	|  Bytes per pixel |   	|
|:-:	|---	|---	|---	|---	|---	|
|  Float 32 bit 	|  np.float32 	|   gdal.GDT_Float32	|   6	|   4	|
|  Unsigned Integer 16 bit	|  np.uint16	|   gdal.GDT_UInt16	    |   2 	|   2	|
|  Unsigned Integer 8 bit 	|  np.uint8 	|   gdal.GDT_Byte   	|   1	|   1	|

Compare the result of the computation with the available RAM on the computer running the notebook.

### Reading the SAR Time Series Subset
Let's read a image subset (offset 2000, 2000 /  size 1024, 1024) of the entire time series data stack. The data are representatios of linearly scaled amplitudes scaled to unsigned 16 bit interger

We use the gdal ReadAsArray(xoff,yoff,xsize,ysize) function where

- xoff = offset in pixels from upper left
- yoff = offset in lines from upper left
- xsize = number of pixels 
- ysize = number of lines

If ReadAsArray() is called without any parameters set, the entire image data stack is read. ReadAsArray() returns a numpy array of the form:

    [bands,lines,pixels]

In [ ]:
# Alternatively you can make subset a tuple variable and use 
# it in the ReadAsArray function call prefixed with a star
subset=(200,200,1000,1000)
rasterDN = img.ReadAsArray(*subset)
rasterDN[rasterDN==0]=1

The numpy .shape object tells us the dimensions of this data stack as *bands (here:time steps), lines, and pixels*:

In [ ]:
rasterDN.shape

## Data conversion from linear scaled amplitudes to dB, power and amplitude data

The values of the raw image data show the linearly scaled amplitude values. These digital number (DN) values need to be converted to proper backscatter values of $\gamma^o$.

We consider conversion to dB scale (logarithmic scale for the expression of the SAR backscatter, power, or amplitude scale.

SAR backscatter data of radiometrically terrain corrected data are often expressed as $\sigma^o$ or the terrain flattened $\gamma^o$ backscattering coefficients. For forest and land cover monitoring applications $\gamma^o$ is the preferred metric. Conversion from power to the logarithmic decibel (dB) scale follows:

$\gamma^o_{dB} = 10 \times log_{10}(\gamma^o_{power})$

As per widely used convention SAR backscatter data are often stored in 16bit unsigned integer values as linearly scaled amplitude data (referred to below as digital numbers **DN**), conversion to dB scale from the linear scaled amplitues is performed with a standard **calibration factor of -83 dB**. This is how ALOS SAR data are distributed by JAXA, how Earth Big Data LLC produces all SAR data including Sentinel-1 data, and how NISAR data will likely be scaled:

$\gamma^o_{dB} = 20 * log10(DN) -83$

In [ ]:
rasterdB=20*np.log10(rasterDN)-83

Conversion from dB to power:

$\gamma^o_{pwr} = 10^{\frac{\gamma^o_{dB}}{10}}$

In [ ]:
rasterPwr=np.power(10.,rasterdB/10.)

Conversion from power to amplitude:

$\gamma^o_{amp} = \sqrt{\gamma^o_{pwr}}$

In [ ]:
rasterAmp=np.sqrt(rasterPwr)

### Explore the image bands of the time steps
Let's explore how a band looks in the various image scales

#### Choose the band number and find which date it is

In [ ]:
bandnbr=20
tindex[bandnbr-1]

Below is the python code to create a four-part figure comparing the effect of the representation of the backscatter values in the DN, amplitude, power and dB scale.

In [ ]:
fig=plt.figure(figsize=(16,16))

ax1=fig.add_subplot(221)
ax2=fig.add_subplot(222)
ax3=fig.add_subplot(223)
ax4=fig.add_subplot(224)

ax1.imshow(rasterDN[bandnbr],cmap='gray',
           vmin=np.percentile(rasterDN,3),
           vmax=np.percentile(rasterDN,97),interpolation='bilinear')
ax2.imshow(rasterAmp[bandnbr],cmap='gray',
           vmin=np.percentile(rasterAmp,3),
           vmax=np.percentile(rasterAmp,97),interpolation='bilinear')
ax3.imshow(rasterPwr[bandnbr],cmap='gray',
           vmin=np.percentile(rasterPwr,3),
           vmax=np.percentile(rasterPwr,97),interpolation='bilinear')
ax4.imshow(rasterdB[bandnbr],cmap='gray',
           vmin=-15,
           vmax=0,interpolation='bilinear')

ax1.set_title('DN Scaled (Amplitudes)')
ax2.set_title('Amplitude Scaled')
ax3.set_title('Power Scaled')
_=ax4.set_title('dB Scaled')

### Comparing histograms of the amplitude, power, and dB scaled data

In [ ]:
# Setup for three part figure
fig=plt.figure(figsize=(16,4))
fig.suptitle('Comparison of Histograms of SAR Backscatter in Different Scales',fontsize=14)
ax1=fig.add_subplot(131)
ax2=fig.add_subplot(132)
ax3=fig.add_subplot(133)

# Important to "flatten" the 2D raster image to produce a historgram
ax1.hist(rasterAmp[bandnbr].flatten(),bins=100,range=(0.,0.7))
ax2.hist(rasterPwr[bandnbr].flatten(),bins=100,range=(0.,0.5))
ax3.hist(rasterdB[bandnbr].flatten(),bins=100,range=(-25,0))

# Means, medians and stddev
amp_mean=rasterAmp[bandnbr].mean()
amp_std=rasterAmp[bandnbr].std()
pwr_mean=rasterPwr[bandnbr].mean()
pwr_std=rasterPwr[bandnbr].std()
dB_mean=rasterdB[bandnbr].mean()
dB_std=rasterdB[bandnbr].std()

# Some lines for mean and median
ax1.axvline(amp_mean,color='red')
ax1.axvline(np.median(rasterAmp[bandnbr]),color='blue')
ax2.axvline(pwr_mean,color='red',label='Mean')
ax2.axvline(np.median(rasterPwr[bandnbr]),color='blue',label='Median')
ax3.axvline(dB_mean,color='red')
ax3.axvline(np.median(rasterdB[bandnbr]),color='blue')

# Lines for 1 stddev
ax1.axvline(amp_mean-amp_std,color='gray')
ax1.axvline(amp_mean+amp_std,color='gray')
ax2.axvline(pwr_mean-pwr_std,color='gray',label='1 $\sigma$')
ax2.axvline(pwr_mean+pwr_std,color='gray')
ax3.axvline(dB_mean-dB_std,color='gray')
ax3.axvline(dB_mean+dB_std,color='gray')

ax1.set_title('Amplitude Scaled')
ax2.set_title('Power Scaled')
ax3.set_title('dB Scaled')
_=ax2.legend()


## Why is the scale important?
It is critical to use the correct scaleing of SAR data for image processing operations. As we can see from the comparison of the histograms, the amplitude, power, or dB scales have different statistial distributions. 

In time series analysis we often compare measurements at any given time steps against the mean of the time series and compute its residuals. When we compute the mean of observations, it makes a difference wheather we do that in power or dB scale. Since dB scale is a logarithmic scale, we cannot simply average data in that scale. Consider the following backscatter values and their mean:


$\gamma^o_1 = -10 dB$  

$\gamma^o_2 = -15 dB$

Let's compute the mean of these values in power and dB scale and compare the result in dB scale:

In [ ]:
g1_dB = -10
g2_dB = -15
g1_pwr = np.power(10.,-10/10.)
g2_pwr = np.power(10.,-15/10.)

mean_dB = (g1_dB+g2_dB)/2.
mean_pwr = (g1_pwr+g2_pwr)/2.
mean_pwr_inDB = 10.*np.log10(mean_pwr)

print('Mean averaging dB values         : {:.1f}'.format(mean_dB))
print('Mean averagin power values in dB : {:.1f}'.format(mean_pwr_inDB))

As one can see, there is a 0.7 dB difference in the average of these two $\gamma^o$ backscatter values.
If we make mean estimates of backscatter values, the **correct scale** in which operations need to be performed **is the power scale.**
This is critical, e.g. when speckle filters are applied, spatial operations like block averaging are performed, or time series are analyzed. Very often we implement models that relate backscatter to biophysical variables like biomass, forest height, or use thresholds to determine change. Ensure that the proper scaling is done when working with the SAR data applying these models.

Another example of the effects can be illustrated with our backscatter data from the images we extracted. Consider a 1 hectare window extracted from our data sets with an off set of 500, 500 for band 20. We compute the mean over time and space of all the pixels.

In [ ]:
offset=500
size=5
o1=offset
o2=offset+size

In [ ]:
mean_dB = rasterdB[:,o1:o2,o1:o2].mean()
mean_dB

In [ ]:
mean_pwr = rasterPwr[:,o1:o2,o1:o2].mean()
mean_pwr_in_dB = 10.* np.log10(mean_pwr)
mean_pwr_in_dB

As one can see, a difference of close to -3 dB is found simply by operating in the different scales. Hence: CAUTION!

## Exploring Polarization Differences
We look at the backscatter characteristics in SAR data from like polarized (same transmit and reveive polarzation, hh or vv) and cross-polarized (vh or hv polarization). For this, we read a timestep in both polarizations, plot the histograms, and display the images in dB scale. First, we open the images, pick the bands from the same acquisition date, read the raster bands and convert them to dB scale.

In [ ]:
# Open the Images
img_like=gdal.Open(imagefile)
img_cross=gdal.Open(imagefile_cross)
# Pick the bands, read rasters and convert to dB
bandnbr_like=20
bandnbr_cross=20
rl=img_like.GetRasterBand(bandnbr_like).ReadAsArray()
rl[rl==0]=1
rc=img_cross.GetRasterBand(bandnbr_cross).ReadAsArray()
rc[rc==0]=1
rl_dB=20.*np.log10(rl)-83
rc_dB=20.*np.log10(rc)-83

Now, we explore the differences in the polarizations by plotting the images with their histograms. We look at the db ranges over which the histograms spread, and can adjust the linear scaling in the image display accordingly to enhace contrast. In the case below

- C-vv like polarized data are mostly spread from -17.5 to -5 dB
- C-vh cross polarized data are mostly spread from -25 to -10 dB

Thus, we note that the cross-polarized data exhibit a larger dynamic range of about 2.5 dB

In [ ]:
fig,ax=plt.subplots(nrows=2,ncols=2,figsize=(16,16))
fig.suptitle('Comaprison of Like- and Cross-Polarized Sentinel-1 C-band Data',
             fontsize=14)
ax[0][0].set_title('C-VV Image')
ax[0][1].set_title('C-VH Image')
ax[1][0].set_title('C-VV Histogram')
ax[1][1].set_title('C-VH Histogram')
ax[0][0].axis('off')
ax[0][1].axis('off')
ax[0][0].imshow(rl_dB,vmin=-15,vmax=0,cmap='gray',interpolation='bilinear')
ax[0][1].imshow(rc_dB,vmin=-20,vmax=-5,cmap='gray',interpolation='bilinear')
ax[1][0].hist(rl_dB.flatten(),range=(-25,0),bins=100)
ax[1][1].hist(rc_dB.flatten(),range=(-25,0),bins=100)
fig.tight_layout()  # Use the tight layout to make the figure more compact